In [11]:
from EvOAutoML import regression

In [12]:
import psutil
import time
import json

In [13]:
from capymoa.stream import stream_from_file
from capymoa.evaluation import RegressionEvaluator, RegressionWindowedEvaluator

In [14]:
import warnings
warnings.filterwarnings("ignore")

In [15]:
datasets = [
    'ailerons',
    'elevators',
    'fried',
    'hyperA',
    'abalone',
    'bike',
    'House8L',
    'MetroTraffic'
]

In [16]:
dataset_name = 'cpu_activity'

In [17]:
stream = stream_from_file(f"RDatasets/{dataset_name}.arff")

In [18]:
regressionEvaluator = RegressionEvaluator(schema=stream.get_schema())
regressionWindowedEvaluator = RegressionWindowedEvaluator(schema=stream.get_schema(),window_size=1000)

In [19]:
EBR = regression.EvolutionaryBaggingRegressor(seed=42)

In [ ]:
t=0
times = []
memories = []
while stream.has_more_instances():
    instance = stream.next_instance()
    x = dict(enumerate(instance.x))
    mem_before = psutil.Process().memory_info().rss # Recording Memory
    start = time.time()  # Recording Time
    try:
        prediction = EBR.predict_one(x)
    except:
        prediction = 0.0
    #print(f"y_true: {instance.y_value}, y_pred: {prediction}")
    regressionEvaluator.update(instance.y_value, prediction)
    regressionWindowedEvaluator.update(instance.y_value, prediction)
    try:
        EBR.learn_one(x, instance.y_value)
    except:
        pass
    end = time.time()
    mem_after = psutil.Process().memory_info().rss
    iteration_mem = mem_after - mem_before
    memories.append(iteration_mem)
    iteration_time = end - start
    times.append(iteration_time)
    t+=1
    if t%1000==0:
        print(f"Running Instance **{t}**")
        print(f"R2 score - {round(regressionEvaluator.R2(),3)}")
        print(f"RMSE score - {round(regressionEvaluator.RMSE(),3)}")
        print("-"*40)

Running Instance **1000**
R2 score - 0.594
RMSE score - 11.78
----------------------------------------
Running Instance **2000**
R2 score - 0.748
RMSE score - 9.545
----------------------------------------
Running Instance **3000**
R2 score - 0.81
RMSE score - 8.518
----------------------------------------
Running Instance **4000**
R2 score - 0.837
RMSE score - 7.884
----------------------------------------
Running Instance **5000**
R2 score - 0.847
RMSE score - 7.439
----------------------------------------
Running Instance **6000**
R2 score - 0.855
RMSE score - 7.091
----------------------------------------


In [ ]:
regressionEvaluator.metrics_dict()

In [13]:
regressionWindowedEvaluator.metrics_per_window()

,classified instances,mean absolute error,root mean squared error,relative mean absolute error,relative root mean squared error,coefficient of determination,adjusted coefficient of determination
0,1000.0,0.001742,0.002889,0.467302,0.455680,0.792356,0.788330
1,1000.0,0.001785,0.003013,0.418634,0.455086,0.792897,0.788881
2,1000.0,0.002642,0.005068,0.429920,0.558770,0.687776,0.681723
3,1000.0,0.001897,0.003930,0.368165,0.586000,0.656604,0.649947
4,1000.0,0.002297,0.003766,0.497785,0.464363,0.784367,0.780187
5,1000.0,0.002283,0.003515,0.445402,0.475766,0.773647,0.769258
6,1000.0,0.001317,0.001876,0.402249,0.432402,0.813028,0.809404
7,1000.0,0.002081,0.003169,0.494521,0.462912,0.785712,0.781558
8,1000.0,0.002083,0.003128,0.459099,0.529662,0.719458,0.714019
9,1000.0,0.002111,0.003261,0.456559,0.524040,0.725382,0.720058


In [ ]:
# saving results in dict
save_record = {
    "model": 'EAML_REG',
    "dataset": dataset_name,
    "regressionEvaluator": regressionEvaluator.metrics_dict(),
    "windows_scores": regressionWindowedEvaluator.metrics_per_window().to_dict(orient='list'),
    "time": times,
    "memory": memories
}

In [ ]:
file_name = f"{save_record['model']}_{save_record['dataset']}.json"

In [ ]:
file_name

In [ ]:
# To store the dictionary in a JSON file
with open(f"TEMP/{file_name}", 'w') as json_file:  # change temp to  saved_results_json for final run
    json.dump(save_record, json_file)